In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/resultado-de-encuestas-2017-2018.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/resultado-de-encuestas-2017-2018.csv?token=AA4GFHO3EKGWGILBCV4BISC6WR6B4
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Pandas 2 - Split, Apply, Combine

<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[Dataset](#section_dataset)

[Problema](#section_problema)

[GroupBy](#section_groupby)

[Agregaciones simples](#section_agregaciones_simples)

[¿Cómo construimos grupos?](#section_construccion_grupos)

$\hspace{.5cm}$[DataFrameGroupBy](#section_dataframegroupby)

$\hspace{.5cm}$[Groupby con diccionarios y series como key](#section_groupby_dict_series)

$\hspace{.5cm}$[Groupby con funciones](#section_groupby_func)

[¿Qué operaciones podemos hacer sobre grupos?](#section_operaciones_grupos)

$\hspace{.5cm}$[Estadística descriptiva sobre grupos](#section_estadistica_grupos)

$\hspace{.5cm}$[Indices jerárquicos](#section_indices_jerarquicos)

$\hspace{.5cm}$[Aggregate, transform, filter](#section_aggregate_transform_filter)

$\hspace{1cm}$[Transformación](#section_transformacion)

$\hspace{1cm}$[Filtro](#section_filtro)

$\hspace{1cm}$[Apply](#section_apply)


---


<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)


Separar un conjunto de datos en categorías, y aplicar una función a cada grupo, que puede ser de agregación, transformación o filtro, es un paso muy frecuente en un flujo de trabajo de análisis de datos.

Después de cargar y preparar un conjunto de datos, es posible que debamos calcular estadísticas de grupo o posiblemente tablas dinámicas para generar informes o visualizaciones.

`pandas` provee métodos que nos permiten realizar estas tareas de forma natural.

En estas guías aprenderemos a 

* Dividir un objeto `pandas` en partes usando una o más keys

* Calcular medidas de resúmen sobre grupos, como cantidad, media, desvío estandar, o cualquier función definida por el usuario

* Aplicar transformaciones por grupos.

* Construir tablas pivot


<a id="section_dataset"></a> 
## Dataset

[volver a TOC](#section_toc)


El GCBA realiza encuestas a los turistas que se acercan a los centros de atención. Se pregunta el motivo de la consulta, los días que dura el viaje, el país de origen, entre otras cosas.

El dataset es de acceso público en el portal de datos abiertos del GCBA.

https://data.buenosaires.gob.ar/dataset/encuesta-centros-atencion-turistica-cat

En esta guía vamos a usar el dataset Resultado de encuestas en Centros de Atención Turística (CAT) en 2017-2018

https://data.buenosaires.gob.ar/dataset/encuesta-centros-atencion-turistica-cat/archivo/juqdkmgo-942-resource    

---
<a id="section_problema"></a> 
## Problema

[volver a TOC](#section_toc)

Usando los datos de las consultas en las oficinas de turismo de la Ciudad de Buenos Aires, vamos a responder preguntas sobre el país de orígen de los turistas, algunas medidas estadísticas sobre cantidad de días de estadía en la ciudad y cantidad de visitantes.


---

<a id="section_groupby"></a> 
## GroupBy

[volver a TOC](#section_toc)



Podemos describir las operaciones sobre grupos con el término *split-apply-combine*

En la primera etapa del proceso, los datos en un objeto `pandas` (una instancia de `Series` o de `DataFrame`) se dividen en grupos (*split*) en base a una o más keys que definimos. Esta división se lleva a cabo por filas (axis = 0) o por columnas (axis = 1).

Como segunda etapa, aplicamos una función a cada uno de los grupos (*apply*) dando como resultado un nuevo valor por grupo.

Como último paso, los resultados de la aplicación de la función en cada uno de los grupos se combina en un objeto resultado (*combine*).


![Image](img/split_apply_combine.png)


La claves por las que agrupamos pueden especificarse de varias formas distintas:

* Una lista o numpy array del mismo tamaño que el eje seleccionado

* Para objetos DataFrame, un string que indica el nombre de columna por la que vamos a agrupar.

* Para objetos DataFrame, un string que indica el nombre de index por el que vamos a agrupar.

* Un diccionario o Series que establezca un mapeo entre un valor y el nombre del grupo

* Una función python que se evaluará en cada una de las etiquetas del eje

* Una lista con cualquiera de las opciones de arriba.


Observemos que el resultado de cada una de esas opciones es **producir un array de valores que usaremos para dividir** el objeto Series o DataFrame


---

Vamos a leer los datos de la encuesta de turismo del GCBA, ver de qué tamaño es el DataFrame, qué columnas tiene, de qué tipo de datos son, y los primeros registros leídos.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).

data = pd.read_csv("../Data/resultado-de-encuestas-2017-2018.csv", sep = ",", low_memory=False) 
print(data.shape)
print(data.columns)
print(data.dtypes)
data.head(3)

Veamos qué pocentaje de registros null hay en cada columna

In [ ]:
data.isnull().sum() / data.shape[0]

Vemos que hay varias columnas con un porcentaje muy alto de valores nulos.

---

<a id="section_agregaciones_simples"></a> 
## Agregaciones simples

[volver a TOC](#section_toc)




### Pregunta 1:

La columna pernoctaciones es de tipo numérico (float64), calculemos cuántos días en total y en promedio dura el viaje de las personas que se acercaron a estos centros de atención.


In [ ]:
pernoctaciones_serie = data.pernoctaciones
print("duración promedio del viaje", pernoctaciones_serie.mean().round(2))
print("duración total del viaje", pernoctaciones_serie.sum())

### Pregunta 2:

**2.a** ¿Cuántos y cuáles son los motivos de consulta?

Vamos a analizar el campo `motivo_consulta` que no tiene valores nulos.

Ayuda: `value_counts`
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

**2.b** ¿Cuáles son los 5 motivos más consultados?

Ayuda: Indexemos la serie ordenda que es resultado de value_counts 

(value_counts devuelve la serie ordenada).



In [ ]:
motivos = data.motivo_consulta.value_counts()
motivos.head(3)

In [ ]:
motivos_top5 = motivos[0:5]
motivos_top5

### Pregunta 3:

Usando el método `describe` evaluemos las columnas pasajeros y pernoctaciones

Este método nos devuelve un DataFrame, usando ese resultado respondamos cuál es la media de cantidad de pasajeros.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html

In [ ]:
medidas = data[["pasajeros", "pernoctaciones"]].describe()
print(type(medidas))
print("media cantidad de pasajeros: ", medidas.loc["mean", "pasajeros"].round(2))
medidas

Vemos que el máximo de pernoctaciones es 690 y el máximo de pasajeros es 150. 

Veamos qué registros tienen estos valores. Y tratemos de entender si corresponden o no a un error.

In [ ]:
data_pasajeros_150_mask = data.pasajeros == 150
data_pasajeros_150 = data.loc[data_pasajeros_150_mask, ]
data_pasajeros_150

Parece ser un contingente de estudiantes franceses en tránsito ("No pernocta en Buenos Aires").

In [ ]:
data_pernoctaciones_690_mask = data.pernoctaciones == 690
data_pernoctaciones_690 = data.loc[data_pernoctaciones_690_mask, ]
data_pernoctaciones_690

De este registro no podemos extraer info que justifique 690 días de estadía.

<a id="section_construccion_grupos"></a> 
##  ¿Cómo construimos grupos?

[volver a TOC](#section_toc)



---

<a id="section_dataframegroupby"></a> 
### DataFrameGroupBy

[volver a TOC](#section_toc)


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

La definición abstracta de agrupación es proporcionar un mapeo entre valores y (etiquetas o) nombres de grupos.

Un objeto DataFrameGroupBy no calcula nada, sino que crea una estructura intermedia con toda la información necesaria para luego aplicar alguna operación a cada grupo.

El **resultado de esa operación** es devuelto en una Series o DataFrame **indexado por los valores únicos de la clave del groupby**.

Las operaciones que podemos aplicar sobre un objeto DataFrameGroupBy están listadas aquí
https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html

Usando como ejemplo el resultado de una operación groupby por nombre de columna, vamos a presentar algunas properties y métodos de este objeto.

Agrupemos los datos por la columna `pais_residencia_si_extranjero`.

#### Tipo 
Veamos de qué tipo es el objeto devuelto



In [ ]:
data_grouped = data.groupby('pais_residencia_si_extranjero')
type(data_grouped)

#### size

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.size.html

Veamos la cantidad de registros en cada grupo y cuántos registros del DataFrame original están asignados a algun grupo.

In [ ]:
data_grouped.size()

In [ ]:
data_grouped.size().sum()

In [ ]:
data_grouped.size().sum() / data.shape[0]

Sólo el 52% de los registros fue asignado a un grupo.

Veamos que esa cantidad de registros asignados a un grupo coincide con la cantidad de registros no nulos en ese campo. (groupby no arma un grupo de key null).

In [ ]:
data.loc[data.pais_residencia_si_extranjero.notnull(), ].shape[0]

#### Indices

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.indices.html#pandas.core.groupby.GroupBy.indices

Es un diccionario cuyas claves son los valores únicos de las claves del groupby, en este caso los valores de la columna pais_residencia_si_extranjero, y cuyos valores son un array con los índices del DataFrame cuyo valor en esa columna es igual al de la clave.


In [ ]:
print(type(data_grouped.indices))
data_grouped.indices

En este ejemplo, los índices 48, 108, 147, ... de data corresponden a Alemania; los índices 33, 242, 267, ... corresponden a Australia, los índices 6, 8, 28, ... a Uruguay. Verifiquemos esto:

In [ ]:
data.pais_residencia_si_extranjero.loc[[48, 108, 147]]

In [ ]:
data.pais_residencia_si_extranjero.loc[[33, 242, 267]]

In [ ]:
data.pais_residencia_si_extranjero.loc[[ 6, 8, 28]]

#### groups

Es similar a `indices` pero asociando las claves del groupby a objetos de tipo Index

In [ ]:
data_grouped.groups

---

<a id="section_groupby_dict_series"></a> 
### Groupby con diccionarios y series como key

[volver a TOC](#section_toc)


Hasta ahora vimos cómo definir una o varias columnas de un DataFrame como clave del groupby. Cuando hacemos esto estamos definiendo para cada registro una etiqueta (el valor del campo o campos key en ese registro) que usaremos para determinar a qué grupo pertenece.

A continuación veremos ejemplos de Series y diccionarios como claves del groupby.

Para poder agrupar usando como key una instancia de Series o diccionario necesitamos que los valores del índice de la serie o las keys del diccionario sean los mismos que los del índice del DataFrame sobre el que queremos agrupar.

Definamos un diccionario que asocie los países con su contienente:  

In [ ]:
pais_en_continente = {
    'Chile': 'America', 'Francia': 'Europa', 'México': 'America', 'Colombia': 'America', 
    'Uruguay': 'America', 'Estados Unidos': 'America', 'España': 'Europa', 'Italia': 'Europa', 
    'India': 'Asia', 'Ecuador': 'America', 'Brasil': 'America',
    'Australia': 'Oceania', 'Bolivia': 'America', 'Reino Unido': 'Europa', 
    'Alemania': 'Europa', 'Israel': 'Asia', 'China': 'Asia', 
    'Venezuela': 'America', 'Países Bajos': 'Europa', 'Canadá': 'America', 'Suiza': 'Europa', 'Turquía': 'Europa',
    'Noruega': 'Europa', 'Corea del Sur': 'Asia', 'Polonia': 'Europa', 'Perú': 'America', 'Paraguay': 'America',
    'Costa Rica': 'America', 'Japón': 'Asia', 'Marruecos': 'Africa', 'Bélgica': 'Europa', 'Malasia': 'Asia', 
    'Rusia': 'Europa', 'Sudáfrica': 'Africa', 'Nueva Zelanda': 'Oceania'        
}

Asignemos como índice del DataFrame los valores del campo pais_residencia_si_extranjero, que coinciden con las keys del diccionario que definimos arriba.

In [ ]:
data.index = data.pais_residencia_si_extranjero

Contemos cuantos turistas provenientes de cada continenente solictaron informes en la oficina de turismo.

In [ ]:
data_grouped_continente = data.groupby(pais_en_continente)
data_grouped_continente["pasajeros"].sum()

De forma similar podemos indexar Series con diccionarios:

In [ ]:
# creamos la serie
serie_pasajeros = data.pasajeros
# asignamos como indice el valor del campo  pais_residencia_si_extranjero para ese registro
serie_pasajeros.index = data.pais_residencia_si_extranjero
# agrupamos y sumamos
serie_pasajeros.groupby(pais_en_continente).sum()

Ahora queremos indexar un DataFrame con un objeto Series.

Transformemos el diccionario pais_en_continente en una instancia de Series, y usemoslo para indexar

In [ ]:
pais_en_continente_serie = pd.Series(pais_en_continente)
pais_en_continente_serie

In [ ]:
data.index = data.pais_residencia_si_extranjero
data_grouped_continente_2 = data.groupby(pais_en_continente_serie)
data_grouped_continente_2["pasajeros"].sum()

<a id="section_groupby_func"></a> 

## Groupby con funciones

[volver a TOC](#section_toc)

Cualquier función que pasemos como key de groupby será invocada una vez por cada valor del índice, y el resultado será el nombre del grupo.

Veamos un ejemplo:

Definimos una función que dado un string que representa un país, devuelve el nombre del contienente de ese país

In [ ]:
def get_continente(pais):
    pais_en_continente = {
    'Chile': 'America', 'Francia': 'Europa', 'México': 'America', 'Colombia': 'America', 
    'Uruguay': 'America', 'Estados Unidos': 'America', 'España': 'Europa', 'Italia': 'Europa', 
    'India': 'Asia', 'Ecuador': 'America', 'Brasil': 'America',
    'Australia': 'Oceania', 'Bolivia': 'America', 'Reino Unido': 'Europa', 
    'Alemania': 'Europa', 'Israel': 'Asia', 'China': 'Asia', 
    'Venezuela': 'America', 'Países Bajos': 'Europa', 'Canadá': 'America', 'Suiza': 'Europa', 'Turquía': 'Europa',
    'Noruega': 'Europa', 'Corea del Sur': 'Asia', 'Polonia': 'Europa', 'Perú': 'America', 'Paraguay': 'America',
    'Costa Rica': 'America', 'Japón': 'Asia', 'Marruecos': 'Africa', 'Bélgica': 'Europa', 'Malasia': 'Asia', 
    'Rusia': 'Europa', 'Sudáfrica': 'Africa', 'Nueva Zelanda': 'Oceania'}
    if pais in pais_en_continente:
        result = pais_en_continente[pais]
    else:
        result = "desconocido"
    return result
    

Repetimos el ejercicio anterior agrupando con esta función.

Recordemos que **la función que es la clave del groupby recibe como argumento el valor del índice de cada registro** cuando axis = 0 (que es el valor por default de axis) y recibe el valor de columna cuando axis = 1

Nosotros en todos los ejercicios de esta práctica agrupamos por filas (axis=0) pero la misma lógica vale si queremos agrupar por columnas.

In [ ]:
data.index = data.pais_residencia_si_extranjero
data_grouped_func = data.groupby(get_continente, axis=0)
data_grouped_func["pasajeros"].sum()

---
<a id="section_operaciones_grupos"></a> 
## ¿Qué operaciones podemos hacer sobre grupos?

[volver a TOC](#section_toc)


In [ ]:
# reseteamos el index de data, que modificamos en los ejercicios anteriores
data = data.reset_index(drop=True)
data_grouped = data.groupby('pais_residencia_si_extranjero')

<a id="section_estadistica_grupos"></a> 
### Estadística descriptiva sobre grupos

[volver a TOC](#section_toc)

https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats

Calculemos la cantidad de turistas de cada país que solicitaron información.

Para esos vamos a seleccionar la columna "pasajeros" del objeto DataFrameGroupBy y sumar ese campo en los registros que componen cada grupo.

In [ ]:
sum_por_pais = data_grouped["pasajeros"].sum()
print(type(sum_por_pais))
sum_por_pais

Vemos que el resultado es un objeto de tipo Series y su indice son los valores únicos del campo que usamos como key del groupby.

Ahora queremos ver un ranking de paises basado en la cantidad de turistas que visitan la Ciudad de Bueos Aires. Para eso ordenamos la serie resultado del punto anterior de mayor a menor, usando el método `sort_values`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.sort_values.html


In [ ]:
sum_por_pais_sorted = sum_por_pais.sort_values(ascending=False)
sum_por_pais_sorted

Calculemos ahora la media y desvío estandar en pernoctaciones por país.

Ya calculamos groupby por pais y lo asignamos a la variable data_grouped, seleccionamos el campo pernoctaciones y calculamos esas medidas

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.mean.html#pandas.core.groupby.GroupBy.mean

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html#pandas.core.groupby.GroupBy.std

También podemos usar `describe` sobre los grupos

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.describe.html

In [ ]:
data_grouped["pernoctaciones"].mean()

In [ ]:
data_grouped["pernoctaciones"].std()

In [ ]:
data_grouped["pernoctaciones"].describe()

<a id="section_indices_jerarquicos"></a> 
### Indices jerárquicos

[volver a TOC](#section_toc)


Podemos agrupar por más de un campo, y el resultado será una Series o DataFrame con un índice jerárquico definido por los campos key del groupby.

Veamos cómo se distribuye la cantidad de turistas por país de residencia por barrio.

In [ ]:
data_grouped_pais_barrio = data.groupby(["pais_residencia_si_extranjero", "barrio"])
cant_pasajeros_pais_barrio = data_grouped_pais_barrio["pasajeros"].sum()
cant_pasajeros_pais_barrio

Vemos que el índice del objeto Series resultado tiene dos niveles. Si queremos ver cómo quedó definido:

In [ ]:
cant_pasajeros_pais_barrio.index

Podemos usar el método `unstack` para crear un DataFrame a partir de este objeto Series

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html

In [ ]:
cant_pasajeros_pais_barrio_df = cant_pasajeros_pais_barrio.unstack()
print(type(cant_pasajeros_pais_barrio_df))
cant_pasajeros_pais_barrio_df.head(3)


<a id="section_aggregate_transform_filter"></a> 
### Aggregate, transform, filter


Una vez que tenemos construidos los grupos (como resultado de la etapa "Split"), en la etapa "Apply" podemos realizar sobre ellos operaciones de:

* **agregación**: cálculo de estadísticas de resumen para cada grupo. Por ejemplo, sum o mean

* **transformación**: cálculos específicos por grupos devolviendo nuevos objetos indexados del mismo modo. Por ejemplo, llenar los NA dentro de un grupo con un valor calculado sobre ese grupo como pueden ser la media, mediana, máximo, etc.

* **filtro**: descartar algunos grupos de acuerdo a algún cálculo sobre el grupo que devuelva True o False. Por ejemplo, descartar los grupos con pocos miembros.






Ejemplos de operaciones de agregación son todos los que vimos hasta ahora. Una vez que construimos un grupo con alguna de las alternativas que presentamos, calculamos una medida sobre cada uno de esos grupos.

<a id="section_transformacion"></a> 
#### Transformación

[volver a TOC](#section_toc)

https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#transformation

Vimos que en el campo pernoctaciones hay un porcentaje muy alto de nulos.

Vamos completar los valores de este campo, asignando la media de pernoctaciones agrupando por pais_residencia_si_extranjero y barrio.

Sabemos que groupby no arma grupos definidos por nulos, por lo tanto aquellos registros que tengan null en los campos que son clave de groupby no serán asignados a ningún grupo.

Antes de empezar quitemos estos registros.


In [ ]:
data_key_not_null_mask = np.logical_and(data.pais_residencia_si_extranjero.notnull(), data.barrio.notnull())
data_key_not_null = data.loc[data_key_not_null_mask, :]
data_key_not_null.shape

In [ ]:
# si es necesario, eliminamos el indice que asignamos para los ejerccios de groupby por serie o diccionario:
data_key_not_null = data_key_not_null.reset_index(drop = True)
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
#data_key_not_null.head(3)

In [ ]:
data_key_not_null_grouped_pais_barrio = data_key_not_null.groupby(["pais_residencia_si_extranjero", "barrio"])

¿Qué porcentaje de valores nulos hay en la columna pernoctaciones de data_key_not_null?

In [ ]:
data_key_not_null["pernoctaciones"].isnull().sum() / data_key_not_null.shape[0]

Usamos `transform` para completar los valores nulos con la media por grupo, y contamos cuántos valores nulos quedan

In [ ]:
data_filled = data_key_not_null_grouped_pais_barrio["pernoctaciones"].transform(lambda grp: grp.fillna(grp.mean()))
data_filled

In [ ]:
data_filled.isnull().sum()

Vemos que quedan 10 registros que fueron asignados a un grupo, pero siguen siendo nulos. **¿Qué pasó?**

Miremos cuáles son esos registros y qué valores tienen en los campos "pais_residencia_si_extranjero", "barrio"

In [ ]:
data_not_filled = data_filled.loc[data_filled.isnull()]

In [ ]:
data_key_not_null.loc[data_not_filled.index, [ "pais_residencia_si_extranjero", "barrio"]]

Miremos los valores en el campo "pernoctaciones" de los registros de los grupos 
* Malasia	RECOLETA	
* Marruecos	PALERMO	
* Estados Unidos	SIN IDENTIFICAR	
* Brasil	SIN IDENTIFICAR

In [ ]:
malasia_recoleta_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Malasia', 
                                        data_key_not_null.barrio == "RECOLETA")

data_key_not_null.loc[malasia_recoleta_mask, "pernoctaciones"]

In [ ]:
marruecos_palermo_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Marruecos', 
                                        data_key_not_null.barrio == "PALERMO")

data_key_not_null.loc[marruecos_palermo_mask, "pernoctaciones"]

In [ ]:
eeuu_sinid_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Estados Unidos', 
                                        data_key_not_null.barrio == "SIN IDENTIFICAR")

data_key_not_null.loc[eeuu_sinid_mask, "pernoctaciones"]

In [ ]:
brasil_sinid_mask = np.logical_and(data_key_not_null.pais_residencia_si_extranjero  == 'Brasil', 
                                        data_key_not_null.barrio == "SIN IDENTIFICAR")

data_key_not_null.loc[brasil_sinid_mask, "pernoctaciones"]

Vemos que todos los registros de esos grupos tienen valor nulo en el campor pernoctaciones, por lo tanto la media por grupo también es nula y tenemos otra vez nulo como relleno por grupo.

<a id="section_filtro"></a> 
#### Filtro

[volver a TOC](#section_toc)


https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#filtration

El método `filter` devuelve un subconjunto del objeto original.

Supongamos que queremos devolver los registros que correspondan a países con más de 1000 visitas.


In [ ]:
data_group_pais = data.groupby(data.pais_residencia_si_extranjero)
data_group_pais.size()

In [ ]:
data_paises_frecuentes = data_group_pais.filter(lambda grp: grp["pasajeros"].sum() > 1000)

Tamaño antes del filtro (estamos contando también registros que tienen null en pais_residencia_si_extranjero): 

In [ ]:
data.shape

Tamaño después del filtro (sólo registros que no son null en el campo pais_residencia_si_extranjero):

In [ ]:
data_paises_frecuentes.shape

Otra forma de calcular tamaños antes del filtro:

(sólo registros que no son null en el campo pais_residencia_si_extranjero)

In [ ]:
data_group_pais.size().sum()

<a id="section_apply"></a> 
#### Apply

[volver a TOC](#section_toc)


https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply

Podemos evaluar funciones sobre grupos usando `apply`.

Veamos un ejemplo en el que aplicamos el método decribe sobre cada uno de los grupos por país.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.describe.html

In [ ]:
data_group_pais = data.groupby(data.pais_residencia_si_extranjero)
data_group_pais.apply(lambda grp: grp.describe())

---

#### Referencias

Python for Data Analysis. Wes McKinney. Cap 10

https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-grouping
